## Question Three - Stigler again

In [1]:
# Get data
using CSV

# import Stigler's data set
raw = CSV.read("stigler.csv")

# list of food
foods = raw[2:end,1]

# list of nutrients
nutrients = [string(names(raw)[i]) for i=2:length(names(raw))]

# minimum required amount of each nutrient
boundLower = raw[1,2:end]

# data[f,i] is the amount of nutrient i contained in food f 
data = raw[2:end,2:end]

# Note: data is not a matrix, but a dataframe. 
# You can access individual elements as if it was a matrix:
a = data[1,1]   # get the first element 
println("The nutrient ($(nutrients[1])) in food ($(foods[1])) is $a")
typeof(a)       # will tell you that a is a floating point number

boundLower = raw[1,2:end]
minReq = convert(Array, boundLower)
nutriData= convert(Array, data)

The nutrient (Calories (1000)) in food (Wheat Flour (Enriched)) is 44.7


┌ Warning: `convert(::Type{Array}, dfr::DataFrameRow)` is deprecated, use `permutedims(Vector(dfr))` instead.
│   caller = top-level scope at In[1]:26
└ @ Core In[1]:26
┌ Warning: `convert(::Type{Array}, df::AbstractDataFrame)` is deprecated, use `convert(Matrix, df)` instead.
│   caller = top-level scope at In[1]:27
└ @ Core In[1]:27


77×9 Array{Union{Missing, Float64},2}:
 44.7  1411.0   2.0  365.0   0.0  55.4  33.3  441.0    0.0
 11.6   418.0   0.7   54.0   0.0   3.2   1.9   68.0    0.0
 11.8   377.0  14.4  175.0   0.0  14.4   8.8  114.0    0.0
 11.4   252.0   0.1   56.0   0.0  13.5   2.3   68.0    0.0
 36.0   897.0   1.7   99.0  30.9  17.4   7.9  106.0    0.0
 28.6   680.0   0.8   80.0   0.0  10.6   1.6  110.0    0.0
 21.2   460.0   0.6   41.0   0.0   2.0   4.8   60.0    0.0
 25.3   907.0   5.1  341.0   0.0  37.1   8.9   64.0    0.0
 15.0   488.0   2.5  115.0   0.0  13.8   8.5  126.0    0.0
 12.2   484.0   2.7  125.0   0.0  13.9   6.4  160.0    0.0
 12.4   439.0   1.1   82.0   0.0   9.9   3.0   66.0    0.0
  8.0   130.0   0.4   31.0  18.9   2.8   3.0   17.0    0.0
 12.5   288.0   0.5   50.0   0.0   0.0   0.0    0.0    0.0
  ⋮                                ⋮                      
 13.5   104.0   2.5  136.0   4.5   6.3   1.4   24.0  136.0
 20.0  1367.0   4.2  345.0   2.9  28.7  18.4  162.0    0.0
 17.4  1055.0   3

In [2]:
using JuMP, Clp

original = Model(solver=ClpSolver())
a = @variable(original, moneySpent[1:size(foods,1)] >= 0) # Money spent on each food
b = @variable(original, costFood[1:size(foods,1), 1:size(nutrients,1)]) # Nutrition cost

# Get the nutrition cost
for i in 1:size(foods,1)
    @constraint(original, costFood[i,:] .== nutriData[i,:]*moneySpent[i])
end

# Nutient needs to be >= zero
for j in 1:size(nutrients,1)
    @constraint(original, sum(costFood[:,j]) >= minReq[j])
end
    
@objective(original, Min, sum(moneySpent))     # minimize the price

println(status)
println()

status = solve(original)

println("Objective = ", getobjectivevalue(original) * 365, " is the cost per year." )
println()

println("Normal diet formulation:")
for i in 1:size(foods,1)
    if(getvalue(moneySpent[i]) > 0)
        println(foods[i]," is in the optimal diet")
    end
end

## retrieves the cost of each food based on per nutrient
## for i in 1:size(foods,1)
##     println(getvalue(costFood[i,:]))
## end

UndefVarError: UndefVarError: status not defined

#### Part A : We begin by trying to figure out the cost of each food in relation to our total money spent. This gives us some data to work with moving forward to solve how much should we pay for for the supplements.

In [4]:
using JuMP, Clp

dualModel = Model(solver=ClpSolver())
@variable(dualModel, λ[1:size(foods,1)] >= 0) # money spent on each food

for i in 1:size(nutrients,1)
    @constraint(dualModel, sum(λ'*nutriData[:,i]) >= minReq[i])
end
@objective(dualModel, Min, sum(λ))
status = solve(dualModel)
println(status)
println()
println("Objective = ", getobjectivevalue(dualModel) * 365, " is the cost per year." )
println()
for i in 1:size(foods,1)
    if(getvalue(λ[i]) > 0)
        println(foods[i]," makes up the optimal diet cost ", round(getvalue(λ[i])*365)," per year")
    end
end

Optimal

Objective = 39.66173154546625 is the cost per year.

Wheat Flour (Enriched) makes up the optimal diet cost 11.0 per year
Liver (Beef) makes up the optimal diet cost 1.0 per year
Cabbage makes up the optimal diet cost 4.0 per year
Spinach makes up the optimal diet cost 2.0 per year
Navy Beans, Dried makes up the optimal diet cost 22.0 per year


##### Using duality, we can determine how much we are spending per year on each food. The next goal was to break that down to how much per nutrient was spent each year. This mark was grossly missed. I am at a loss to say where, why, and how I went wrong with my code below. 

In [5]:
using JuMP, Clp

dualModel = Model(solver=ClpSolver())
@variable(dualModel, λ[1:size(nutrients,1)] >= 0) # money spent on each nutrient per day

for i in 1:size(nutrients,1)
    @constraint(dualModel, sum(λ[i]*nutriData[:,i]) >= minReq[i])
end

@objective(dualModel, Min, sum(λ))
status = solve(dualModel)
println(status)
println(getobjectivevalue(dualModel)*365)
for i in 1:size(nutrients,1)
    if(getvalue(λ[i]) > 0)
        println(getvalue(λ[i,:])," dollars spent on ",nutrients[i])
    end
end

Optimal
11.796000686327215
[0.00404476] dollars spent on Calories (1000)
[0.0034344] dollars spent on Protein (g)
[0.00485437] dollars spent on Calcium (g)
[0.00201613] dollars spent on Iron (mg)
[0.00198389] dollars spent on Vitamin A (1000 IU)
[0.00353635] dollars spent on Thiamine (mg)
[0.00590551] dollars spent on Riboflavin (mg)
[0.00358066] dollars spent on Niacin (mg)
[0.00296173] dollars spent on Ascorbic Acid (mg)


Apparently the money required to spend each year dropped. I really do not know how else to evaluate this problem, and apparently I am still wrong with what I am trying. Regardless, I would say dont spend more than $6.00 on the supplements per year. If the daily requirment is 0.8g of calcium, and we know the values based on spending a single dollar. It would take 292 grams of calcium to meet the minimum requirement for a year (0.8 * 365). It is a near one to one ratio in dollar spent to gram of calcium. Taking 292 and dividing it by 50 (which is how many grams of calcium come in the supplements with respect to a single dollar) would be about 5.84 dollars in supplements to satisfy this. 

There are a lot of assumptions here and my logic is highly flawed, but I am trying to work with what I got. Thank you for your understanding. 

#### Part B

#### We are given a cost for each supplement pill, and then asked for a new optimal diet plan. We need to determine how much we will save. The answer is 39.66 - 36.99 = 2.67.

In [10]:
# Get data
using JuMP, Clp
using CSV

# import Stigler's data set
raw = CSV.read("stiglerWithSupp.csv")

# list of food
foods = raw[2:end,1]

# list of nutrients
nutrients = [string(names(raw)[i]) for i=2:length(names(raw))]

# minimum required amount of each nutrient
boundLower = raw[1,2:end]

# data[f,i] is the amount of nutrient i contained in food f 
data = raw[2:end,2:end]

boundLower = raw[1,2:end]
minReq = convert(Array, boundLower)
nutriData = convert(Array, data)

dualModel = Model(solver=ClpSolver())
@variable(dualModel, λ[1:size(foods,1)] >= 0) # money spent on each food
for i in 1:size(nutrients,1)
    @constraint(dualModel, sum(λ'*nutriData[:,i]) >= minReq[i])
end
@objective(dualModel, Min, sum(λ))
status = solve(dualModel)
println(status)
println(getobjectivevalue(dualModel) * 365)
for i in 1:size(foods,1)
    if(getvalue(λ[i]) > 0)
        println(foods[i]," makes up the optimal diet cost ", round(getvalue(λ[i])*365)," per year")
    end
end

Optimal
36.9982473745081
Wheat Flour (Enriched) makes up the optimal diet cost 24.0 per year
Liver (Beef) makes up the optimal diet cost 3.0 per year
Cabbage makes up the optimal diet cost 4.0 per year
Spinach makes up the optimal diet cost 1.0 per year
Calcium Supplement makes up the optimal diet cost 5.0 per year


┌ Warning: `convert(::Type{Array}, dfr::DataFrameRow)` is deprecated, use `permutedims(Vector(dfr))` instead.
│   caller = top-level scope at In[10]:21
└ @ Core In[10]:21
┌ Warning: `convert(::Type{Array}, df::AbstractDataFrame)` is deprecated, use `convert(Matrix, df)` instead.
│   caller = top-level scope at In[10]:22
└ @ Core In[10]:22
